## Investigating song LYRIC data...

In [1]:
# library imports
import os
import pandas as pd
from os import listdir
from os.path import isfile, join

# data imports
data_path = "/Users/dayoorigunwa/code_base/music_mapping/data/"
allfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
lyric_file = "SpotifySongLyrics.csv"

# sanity check
lyric_df = pd.read_csv(data_path + lyric_file)
df_attrs = list(dir(lyric_df))
lyric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154556 entries, 0 to 2154555
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       object
 1   1       object
dtypes: object(2)
memory usage: 32.9+ MB


In [2]:
# renaming columns... tbd lololol

In [3]:
# check duplicate titles...
set(lyric_df["0"].value_counts().sort_values())

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [4]:
# adding calculated fields... SONG_LENGTH
lyric_df["song_length"] = lyric_df["1"].apply(lambda x: len(x))
lyric_df.sort_values(by="song_length", ascending=True, inplace=True)
lyric_df.head(5)

,0,1,song_length
20757,र,च,1
16633,य,र,1
16632,य,म,1
16631,य,र,1
16630,य,य,1


In [5]:
# adding calculated fields ... LYRIC_TYPES
def get_bracket_idx(lyric_str):
    bracket_count = 0
    bracket_locs = dict()
    for i, char in enumerate(lyric_str):
        if char == "[":
            start_ind = i
        elif char == "]":
            bracket_locs[bracket_count] = (start_ind, i)
            bracket_count += 1
    return bracket_locs


def extract_brackets(row):
    brackets = row["flavor_text_idx"]
    lyric_str = row["1"]
    bracket_list = []
    for key in list(brackets.keys()):
        bracket_list.append(lyric_str[brackets[key][0] : brackets[key][1] + 1])
    return bracket_list


lyric_df["flavor_text_idx"] = lyric_df["1"].apply(lambda x: get_bracket_idx(x))
lyric_df["flavor_text"] = lyric_df.apply(lambda x: extract_brackets(x), axis=1)
lyric_df.head(5)

,0,1,song_length,flavor_text_idx,flavor_text
20757,र,च,1,{},[]
16633,य,र,1,{},[]
16632,य,म,1,{},[]
16631,य,र,1,{},[]
16630,य,य,1,{},[]


#### TL;DR: 
- The majority of the lyrics seem to have been downloaded correctly, next steps are just to remove the punctuation and bracketed typing for each chunch of text... (Leverage text-mining code from hw-5 to Lemmatize the lyrics you've saved)

In [ ]:
# calculated fields continued... SCURBBED LYRICS
def scrub_brackets(row):
    brackets = row["flavor_text"]
    lyric_str = row["1"]
    for br in brackets:
        lyric_str = lyric_str.replace(br, "")
    return lyric_str


lyric_df["song_lyrics"] = lyric_df.apply(lambda x: scrub_brackets(x), axis=1)
lyric_df.head(5)

In [ ]:
# dropping rows without lyrics for now...
lyric_df = lyric_df[lyric_df["song_lyrics"] != ""]
lyric_df.head(5)

#### TODO: import necessary libraries & restart notebook...

In [ ]:
# Packages required for preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer  # for lemmatization
import re  # regular expression package
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download("wordnet")
stops = set(stopwords.words("english"))

In [ ]:
# Saving Narrative for Natural Language Processing Lemmatization
text_content = lyric_df[["song_lyrics"]]

# Pre-process the data
narratives = []
lemmatizer = WordNetLemmatizer()
text_content_list = [row for row in text_content["song_lyrics"]]

for narrative in range(0, len(text_content_list)):
    # PART 1
    # replace punctuation and special characters with empty strings
    narrative = re.sub(r"[\W_]", " ", str(text_content_list[narrative]))

    # replace single letters with an empty string
    narrative = re.sub(r"\s+[a-zA-Z]\s+", " ", narrative)

    # replace single letters preceded by the char: ^ with an empty string
    narrative = re.sub(r"\^[a-zA-Z]\s+", " ", narrative)

    # trim white space to a single white space
    narrative = re.sub(r"\s+", " ", narrative, flags=re.I)

    # byte encoding indicators with empty string
    narrative = re.sub(
        r"^b\s+", "", narrative
    )  # if a review record is in bytes, the corresponding line will have a letter 'b' appended at the start)

    # force all chars in string to lowercase
    narrative = narrative.lower()

    # replace all numbers with a single empty string
    narrative = re.sub(r"[0-9]+", "", narrative)

    # replace all typos & abbreviations with single empty string
    narrative = re.sub(r"yof", "", narrative)

    # PART 2
    # split string into list of strings, using each white space as a delimiter
    narrative = narrative.split()

    # remove stop words from list of words
    narrative = [w for w in narrative if not w in stops]

    # morphological analysis of the words; removing inflections to identify the base meaning
    narrative = [lemmatizer.lemmatize(word) for word in narrative]

    # add formatted review to list of formatted reviews
    narratives.append(narrative)

In [ ]:
# Create edge list
test = narratives
edgelist = []
for i in test:
    for j in range(len(i) - 1):
        for k in range(j + 1, len(i)):
            edgelist.append([i[j], i[k]])

In [ ]:
# create pandas series from list of lists
edge_df = pd.DataFrame(edgelist)

# savings intermediate results for now...
print("Saving cleaned song lyric data... \n")
edge_df.to_csv(data_path + "SpotifySongLyrics.csv", index=False)

In [ ]:
# formatting magic...
edge_series = pd.Series(edgelist)
edge_df = edge_series.to_frame()

In [ ]:
# Convert to dictionary and store weight - using Dask
import dask.dataframe as dd
from dask.dataframe.core import get_parallel_type

# FROM pandas TO dask
dask_df = dd.from_pandas(edge_series, chunksize=128)

# Setting up dask computations
dask_df = dask_df.value_counts()

In [ ]:
# applying Dask computations
edge_count = dask_df.compute(num_workers=12)

In [ ]:
edge_count

In [ ]:
# # Setup the calculation graph; unlike Pandas code, #.value_counts().to_dict()
# # no work is done at this point:
# def get_counts(df):
#     by_party = df.groupby("Party Affiliation ")
#     street = by_party["Residential Address Street Name "]
#     return street.value_counts()
# result = get_counts(df)

# # Actually run the computation, using 2 threads:
# result = result.compute(num_workers=2)

# # Sort using normal Pandas DataFrame, since Dask's
# # Pandas emulation doesn't implement this method:
# result.sort_values(ascending=False, inplace=True)

# print(result)
# weight = pd.Series(edgelist).value_counts().to_dict()